In [33]:
import numpy as np
import pandas as pd
import re

In [34]:
df = pd.read_csv("bi.csv", delimiter=",", encoding='ISO-8859-1')

In [35]:
df.head(20)

,fNAME,lNAME,Age,gender,country,residence,entryEXAM,prevEducation,studyHOURS,Python,DB
0,Christina,Binger,44,Female,Norway,Private,72,Masters,158,59.0,55
1,Alex,Walekhwa,60,M,Kenya,Private,79,Diploma,150,60.0,75
2,Philip,Leo,25,Male,Uganda,Sognsvann,55,HighSchool,130,74.0,50
3,Shoni,Hlongwane,22,F,Rsa,Sognsvann,40,High School,120,NaN,44
4,Maria,Kedibone,23,Female,South Africa,Sognsvann,65,High School,122,91.0,80
5,Hannah,Hansen,25,female,Norge,BI Residence,66,High School,130,88.0,59
6,Ole,Johansen,27,Male,Norway,BI-Residence,90,Bachelors,156,80.0,91
7,Lars,Olsen,29,Male,norway,BIResidence,89,Barrrchelors,160,85.0,60
8,Bjørn,Larsen,31,Male,Norway,BI Residence,88,Bachelors,156,80.0,89
9,Sofie,Jensen,33,Female,Denmark,BI_Residence,85,Bachelors,160,83.0,90


In [36]:
df.dtypes

fNAME             object
lNAME             object
Age                int64
gender            object
country           object
residence         object
entryEXAM          int64
prevEducation     object
studyHOURS         int64
Python           float64
DB                 int64
dtype: object

In [37]:
df.shape

(77, 11)

In [38]:
df2 = df.copy()

In [39]:
df2 = df2.rename(columns={'fNAME': 'FirstName', 'lNAME': 'LastName', 'gender' : 'Gender', 'country' : 'Country', 'residence' : 'Residence', 'entryEXAM' : 'EntryExamScore', 'prevEducation' : 'PreviousEducation', 'studyHOURS' : 'StudyHours'})

In [40]:
df2.columns

Index(['FirstName', 'LastName', 'Age', 'Gender', 'Country', 'Residence',
       'EntryExamScore', 'PreviousEducation', 'StudyHours', 'Python', 'DB'],
      dtype='object')

In [41]:
print("Missing values per column:")
print(df2.isnull().sum())

Missing values per column:
FirstName            0
LastName             0
Age                  0
Gender               0
Country              0
Residence            0
EntryExamScore       0
PreviousEducation    0
StudyHours           0
Python               2
DB                   0
dtype: int64


In [42]:
duplicate_count = df2.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 0


In [43]:
text_cols = ['FirstName', 'LastName', 'Gender', 'Country', 'Residence']
for col in text_cols:
    if df2[col].dtype == 'object':
        # Convert to string, strip whitespace, and convert to title case where appropriate
        df2[col] = df2[col].astype(str).str.strip()
        if col in ['FirstName', 'LastName', 'Country', 'Residence']:
            df2[col] = df2[col].str.title()
        if col == 'Gender':
            df2[col] = df2[col].str.lower()
            # Standardize gender entries
            gender_map = {'m': 'male', 'f': 'female', 'male': 'male', 'female': 'female'}
            df2[col] = df2[col].map(lambda x: gender_map.get(x, x))

In [44]:
df2['Age'] = df2['Age'].clip(lower=16, upper=100)  # Assuming students are at least 16
df2['StudyHours'] = df2['StudyHours'].clip(lower=0, upper=100)

In [45]:
score_cols = ['EntryExamScore', 'Python', 'DB']
for col in score_cols:
    df2[col] = df2[col].clip(lower=0, upper=100)

In [46]:
cat_cols = ['Gender', 'Country', 'PreviousEducation']
for col in cat_cols:
    df2[col] = pd.Categorical(df2[col])

In [47]:
duplicate_count = df2.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

Number of duplicate rows: 0


In [48]:
print("\nCleaned DataFrame Info:")
print(df2.info())


Cleaned DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   FirstName          77 non-null     object  
 1   LastName           77 non-null     object  
 2   Age                77 non-null     int64   
 3   Gender             77 non-null     category
 4   Country            77 non-null     category
 5   Residence          77 non-null     object  
 6   EntryExamScore     77 non-null     int64   
 7   PreviousEducation  77 non-null     category
 8   StudyHours         77 non-null     int64   
 9   Python             75 non-null     float64 
 10  DB                 77 non-null     int64   
dtypes: category(3), float64(1), int64(4), object(3)
memory usage: 6.3+ KB
None


In [49]:
diploma_pattern = re.compile(r'.*diploma.*', re.IGNORECASE)
df2['PreviousEducation'] = df2['PreviousEducation'].apply(
    lambda x: "Diploma" if isinstance(x, str) and diploma_pattern.match(x) else x)

In [50]:
print("\nSample of updated data:")
print(df2[['PreviousEducation']].sample(min(10, len(df2))))


Sample of updated data:
   PreviousEducation
43           Masters
3        High School
22       High School
44           Masters
69         Bachelors
55         Doctorate
21       High School
49       High School
30           Diploma
63       High School


In [51]:
corrections = {
    'Barrrchelors': 'Bachelors',
    'HighSchool': 'High School'
}

def standardize_education(value):
    # First check if it's in our specific corrections
    if isinstance(value, str):
        # Check for specific typos
        if value in corrections:
            return corrections[value]
        
        # Check for diploma variations
        if re.search(r'diploma', value, re.IGNORECASE):
            return "Diploma"
            
    return value

df2['PreviousEducation'] = df2['PreviousEducation'].apply(standardize_education)

In [52]:
print(df2['PreviousEducation'].value_counts())

PreviousEducation
Bachelors      25
High School    19
Masters        16
Diploma        12
Doctorate       5
Name: count, dtype: int64


In [53]:
if 'Rsa' in df2['Country'].values:
    # Replace 'Rsa' with 'RSA'
    df2['Country'] = df2['Country'].replace('Rsa', 'RSA')
    print("\n'Rsa' has been replaced with 'RSA'")
else:
    print("\n'Rsa' not found in the Country column")


'Rsa' has been replaced with 'RSA'


C:\Users\pagre\AppData\Local\Temp\ipykernel_1068\247584442.py:3: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df2['Country'] = df2['Country'].replace('Rsa', 'RSA')


In [54]:
# Function to standardize Bi Residence variations
def standardize_bi_residence(value):
    if isinstance(value, str):
        # Updated pattern to match: Bi Residence, Bi_Residence, Bi-Residence, BiResidence, etc.
        # The pattern now includes hyphen in the character class
        if re.search(r'bi[\s_\-]*residence', value, re.IGNORECASE):
            return "Bi Residence"
    return value

# Apply the standardization function
df2['Residence'] = df2['Residence'].apply(standardize_bi_residence)

print("\nAfter standardization:")
print(df2['Residence'].value_counts())


After standardization:
Residence
Private         33
Bi Residence    32
Sognsvann       12
Name: count, dtype: int64


In [55]:
print("Missing values in score columns:")
print(df2[['Python', 'DB']].isnull().sum())
print("\nPercentage of missing values:")
print(df2[['Python', 'DB']].isnull().mean() * 100)

Missing values in score columns:
Python    2
DB        0
dtype: int64

Percentage of missing values:
Python    2.597403
DB        0.000000
dtype: float64


In [56]:
python_median = df2['Python'].median()
df2['Python'] = df2['Python'].fillna(python_median)
print(f"\nImputed Python missing values with median: {python_median}")


Imputed Python missing values with median: 81.0


In [57]:
df2.head(20)

,FirstName,LastName,Age,Gender,Country,Residence,EntryExamScore,PreviousEducation,StudyHours,Python,DB
0,Christina,Binger,44,female,Norway,Private,72,Masters,100,59.0,55
1,Alex,Walekhwa,60,male,Kenya,Private,79,Diploma,100,60.0,75
2,Philip,Leo,25,male,Uganda,Sognsvann,55,High School,100,74.0,50
3,Shoni,Hlongwane,22,female,RSA,Sognsvann,40,High School,100,81.0,44
4,Maria,Kedibone,23,female,South Africa,Sognsvann,65,High School,100,91.0,80
5,Hannah,Hansen,25,female,Norge,Bi Residence,66,High School,100,88.0,59
6,Ole,Johansen,27,male,Norway,Bi Residence,90,Bachelors,100,80.0,91
7,Lars,Olsen,29,male,Norway,Bi Residence,89,Bachelors,100,85.0,60
8,Bjørn,Larsen,31,male,Norway,Bi Residence,88,Bachelors,100,80.0,89
9,Sofie,Jensen,33,female,Denmark,Bi Residence,85,Bachelors,100,83.0,90


In [58]:
df2.to_csv('clean_data.csv', encoding='utf8')